# SQL Data Cleaning for Mobile Coverage Dataset

### Purpose:

This SQL code is designed to clean and prepare a public dataset of mobile coverage data in Google BigQuery.

The primary goal is data cleaning, which includes creating a new table, normalizing categorical features, handling missing data, and removing outliers from numerical features.

This process is essential for ensuring data quality and reliability for subsequent analysis.

### Import libraries and modules

In [ ]:
import pandas as pd
from google.cloud import bigquery

### Import function: Interactive SQL Query to Pandas DataFrame Converter

In [ ]:
# Import the custom query_df and run_query functions for executing BigQuery queries
from query_functions import query_df  # Execute the query and return the output as a DataFrame
from query_functions import run_query  # Execute the query without returning a DataFrame, used for INSERT, UPDATE, DELETE, etc.

### Datasets and Tables paths to Google BigQuery

In [ ]:
# Catalonian mobile coverage eu (2015-2017) --> mobile_data_2015_2017_cleaned
mobile_data_cleaned = "bq-analyst-230590.project_cat_mobile_coverage_2015_2017.mobile_data_2015_2017_cleaned"

### Creating Table and Columns

This following code creates a **copy of the 'mobile_data_2015_2017' table** by selecting specific columns and saves it as 'mobile_data_2015_2017_cleaned

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query:
query = f"""
CREATE TABLE IF NOT EXISTS `{mobile_data_cleaned}` AS
SELECT
    date,
    hour,
    lat,
    long,
    signal,
    network,
    operator,
    status,
    description,
    net,
    speed,
    satellites,
    precission,
    provider,
    activity,
    downloadSpeed,
    uploadSpeed,
    postal_code,
    town_name,
    position_geom
FROM
  `bigquery-public-data.catalonian_mobile_coverage_eu.mobile_data_2015_2017`
    """

# Execute the query
run_query(query)

    - Preview

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query:
query = f"""
SELECT *
FROM `{mobile_data_cleaned}` 
WHERE
    network IS NOT NULL
    AND postal_code IS NOT NULL
LIMIT 10
    """

# Execute the query
raw_data = query_df(query)

# Display data
raw_data

In [ ]:
raw_data.columns

    - Add 'province' new column

A new column, **province** is being added to the BigQuery dataset to include province values, enhancing the dataset for regional analysis.

This new column is currently empty and will be populated based on the structure of Spanish **postal codes**, with the first two digits determining the province name. If the 'postal_code' doesn't match any of these values, it assigns 'Not defined'. Existing rows with pre-defined province values will not be affected by this update.

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: create and add 'province' column
query = f"""
ALTER TABLE `{mobile_data_cleaned}`
ADD COLUMN province STRING;
    """

# Execute the query
run_query(query)

    - Populate 'province' values

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: populate 'province' values
query = f"""
UPDATE `{mobile_data_cleaned}`
SET province = CASE
  # First two digits condition (LEFT) for postal code using CAST to maintain a consistent string data type.
  WHEN LEFT(CAST(postal_code AS STRING), 2) = '08' THEN 'Barcelona'
  WHEN LEFT(CAST(postal_code AS STRING), 2) = '25' THEN 'Lleida'
  WHEN LEFT(CAST(postal_code AS STRING), 2) = '17' THEN 'Girona'
  WHEN LEFT(CAST(postal_code AS STRING), 2) = '43' THEN 'Tarragona'
  ELSE 'Not defined'
END
WHERE province IS NULL
    """

# Execute the query
run_query(query)

    - Rename columns to avoid confusion and fix typos

In the original dataset, there were three key columns: 'net,' 'network,' and 'operator.' These columns contained information about the type of network (e.g., 4G, 3G, 2G), the network provider (e.g., Movistar, Orange), and the specific operator (e.g., Movistar, Orange, ONO, Lowi) associated with the data.

To avoid potential confusion, especially between the 'net' and 'network' columns, the following SQL code is designed to **rename the 'network' column to 'net_provider.'** This renaming ensures that the purpose and meaning of each column are clear and helps improve the overall clarity of the dataset.

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: rename 'network' for 'net_provider'
query = f"""
ALTER TABLE `{mobile_data_cleaned}`
RENAME COLUMN network TO net_provider
    """

# Execute the query
run_query(query)

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: rename 'network' for 'net_provider'
query = f"""
ALTER TABLE `{mobile_data_cleaned}`
RENAME COLUMN precission TO precision
    """

# Execute the query
run_query(query)

    - Column types

In [ ]:
raw_data.info()

In this case, '**date**' and '**hour**' have been transformed to 'object' types. However, in the original Google BigQuery dataset, their formats are 'date' and 'time,' respectively. 

Just in case you need to update these formats in the original table in BigQuery, here are the required queries.

(Please note that we are displaying this data as a Python DataFrame for reference, but the actual changes must be made in the original BigQuery dataset as we will be directly querying it. We won't be working with this DataFrame for further analysis.)

In [ ]:
# SQL query: in BigQuery,convert 'date' to date format and 'hour' to time format
query = f"""
    # Convert 'date' column to date data type
    UPDATE `{mobile_data_cleaned}`
    SET date = CAST(date AS DATE)
    """
# Execute the query
# run_query(query)


query = f"""
    # Convert 'date' column to date data type
    UPDATE `{mobile_data_cleaned}`
    SET hour = CAST(hour AS TIME)
    """

# Execute the query
# run_query(query)

### Categorical Features

#### Standardize data names

**Net provider**

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: original count of unique net_provider
query = f"""
SELECT
 COUNT(DISTINCT(net_provider)) original_unique_net_provider
FROM `{mobile_data_cleaned}`
    """

# Execute the query
query_df(query)

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: example of net_provider names not standardized (Movistar)
query = f"""
SELECT
 DISTINCT(net_provider)
FROM `{mobile_data_cleaned}`
WHERE
 UPPER(net_provider) LIKE 'MO%IS%'
    """

# Execute the query
query_df(query)

    - Unusual characters

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: unusual characters to take into account
query = f"""
SELECT DISTINCT net_provider
FROM `{mobile_data_cleaned}`
# Filter out net_provider values that contain non-alphanumeric characters (letters, digits, and spaces).
WHERE REGEXP_CONTAINS(net_provider, r'[^a-zA-Z0-9 ]')
    """

# Execute the query
query_df(query)

This following code standardizes the 'net_provider' names in the table. It employs various functions like UPPER (to convert to uppercase), TRIM (to remove leading/trailing spaces), and '=' or LIKE (with % wildcard for pattern matching) to recognize and group similar operator names under a common name.

For example, 'movistar' and 'Mobistar' are both categorized as 'Movistar'. It ensures uniformity in the 'net_provider' column, making the data more consistent and easier to work with.

    - Standardize net_provider names

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: Update net_provider names
query = f"""
UPDATE `{mobile_data_cleaned}`
SET net_provider = CASE
    # WHEN UPPER(TRIM(net_provider)) = '3' THEN 'Three'
    WHEN UPPER(TRIM(net_provider)) LIKE '3%' THEN 'Three'
    #WHEN UPPER(TRIM(net_provider)) = 'BITEL' THEN 'Bytel'
    WHEN UPPER(TRIM(net_provider)) LIKE '%AIRTEL%' THEN 'Airtel'
    WHEN UPPER(TRIM(net_provider)) LIKE '%B%TEL%' THEN 'Bytel'
    WHEN UPPER(TRIM(net_provider)) LIKE '%BOUYGUES%' THEN 'Bouygues Telecom'
    WHEN UPPER(TRIM(net_provider)) LIKE 'BUSCANDO %' THEN 'Sense Servei'
    WHEN UPPER(TRIM(net_provider)) = 'CABLE MOVIL' THEN 'Cable Movil'
    WHEN UPPER(TRIM(net_provider)) = 'CABLEMOVIL' THEN 'Cable Movil'
    WHEN UPPER(TRIM(net_provider)) LIKE 'CLARO%' THEN 'Claro'
    WHEN UPPER(TRIM(net_provider)) LIKE 'CUBACEL%' THEN 'Cubacel'
    WHEN UPPER(TRIM(net_provider)) LIKE 'E-%' THEN 'EE'
    WHEN UPPER(TRIM(net_provider)) LIKE '%EMER%' THEN 'Nomes Trucades Emergencies'
    WHEN UPPER(TRIM(net_provider)) LIKE 'JAZZTEL%' THEN 'Jazztel'
    WHEN UPPER(TRIM(net_provider)) LIKE 'LOWI%' THEN 'Lowi'
    WHEN LEFT(UPPER(TRIM(net_provider)), 14) = 'FRANCE TELECOM' THEN 'France Telcom Espana SA'
    WHEN UPPER(TRIM(net_provider)) LIKE 'MASMOVIL%' THEN 'Masmovil'
    WHEN UPPER(TRIM(net_provider)) LIKE '%MOBILAND%' THEN 'Mobiland'
    WHEN UPPER(TRIM(net_provider)) LIKE 'MOVILNET%' THEN 'Movilnet'
    WHEN UPPER(TRIM(net_provider)) LIKE '%MO%ISTAR%' THEN 'Movistar'
    # WHEN UPPER(TRIM(net_provider)) LIKE 'MOBISTAR%' THEN 'Movistar'
    # WHEN UPPER(TRIM(net_provider)) LIKE '%MOBISTAR%' THEN 'Movistar'
    # WHEN UPPER(TRIM(net_provider)) LIKE 'MOVISTAR%' THEN 'Movistar'
    WHEN UPPER(TRIM(net_provider)) LIKE 'MTS%' THEN 'MTS'
    WHEN UPPER(TRIM(net_provider)) LIKE '%ORANGE%' THEN 'Orange'
    WHEN UPPER(TRIM(net_provider)) = 'A1' THEN 'Orange'
    WHEN UPPER(TRIM(net_provider)) LIKE 'O2%' THEN 'O2'
    WHEN UPPER(TRIM(net_provider)) LIKE 'PE%EPHONE' THEN 'Pepephone'
    # WHEN UPPER(TRIM(net_provider)) = 'PELEPHONE' THEN 'Pepephone'
    # WHEN UPPER(TRIM(net_provider)) = 'PEPEPHONE' THEN 'Pepephone'
    WHEN UPPER(TRIM(net_provider)) = 'PROXIMUS' THEN 'Proximus'
    WHEN UPPER(TRIM(net_provider)) LIKE 'REPUBLICA%' THEN 'Republica Movil'
    WHEN UPPER(TRIM(net_provider)) LIKE 'SENSE %' THEN 'Sense Servei'
    WHEN UPPER(TRIM(net_provider)) LIKE 'SIMYO%' THEN 'Simyo'
    WHEN UPPER(TRIM(net_provider)) LIKE 'SIN %' THEN 'Sense Servei'
    WHEN UPPER(TRIM(net_provider)) LIKE 'TIGO%' THEN 'TIGO'
    WHEN UPPER(TRIM(net_provider)) LIKE 'TIM%' THEN 'TIM'
    WHEN UPPER(TRIM(net_provider)) LIKE 'TELEF%' THEN 'Movistar'
    # WHEN (TRIM(network)) = 'Telefonica Moviles Espana' THEN 'Movistar'
    # WHEN (TRIM(network)) = 'Telef├│nica M├│viles Espa├▒a' THEN 'Movistar'
    WHEN UPPER(TRIM(net_provider)) LIKE 'TDC%' THEN 'TDC Mobile'
    WHEN UPPER(TRIM(net_provider)) LIKE 'TELEKOM%' THEN 'Telekom'
    WHEN UPPER(TRIM(net_provider)) LIKE '%TELENOR%' THEN 'Telenor'
    WHEN UPPER(TRIM(net_provider)) LIKE '%T-MOBILE%' THEN 'T-Mobile'
    WHEN UPPER(TRIM(net_provider)) LIKE 'VIVO%' THEN 'Vivo'
    WHEN UPPER(TRIM(net_provider)) LIKE '%VODAFONE%' THEN 'Vodafone'
    WHEN UPPER(TRIM(net_provider)) = 'VF ES' THEN 'Vodafone'
    WHEN UPPER(TRIM(net_provider)) LIKE 'VODACOM%' THEN 'Vodacom'
    WHEN UPPER(TRIM(net_provider)) LIKE 'YOIGO%' THEN 'Yoigo'
    ELSE net_provider
END
WHERE net_provider IS NOT NULL;

    """

# Execute the query
run_query(query)

    - Null values in net_provider

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: Count null values and calculate the percentage
query = f"""
WITH NetProviderCount AS (
 SELECT
   COUNT(*) AS total_count_net_provider
 FROM `{mobile_data_cleaned}`
),
NullProviderCount AS (
 SELECT
   COUNT(*) AS null_count_net_provider
 FROM `{mobile_data_cleaned}`
 WHERE
   net_provider IS NULL
   OR net_provider = 'null'
)

SELECT
 null_count_net_provider,
 CONCAT(ROUND((null_count_net_provider/total_count_net_provider)*100, 2), " %") AS perc_null_net_provider
FROM NetProviderCount, NullProviderCount
"""

# Execute the query
query_df(query)

     - Remove rows with specified net_provider values and NULL net_providers from the table.

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: drop specific rows
query = f"""
DELETE FROM `{mobile_data_cleaned}`
WHERE
 net_provider IS NULL
 OR net_provider IN ('000000', '21303', '21401', '23866', '90118', '?????', 'null')
    """

# Execute the query
run_query(query)

    - Remove rows where the activity count for each net_provider is less than 10

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: drop specific rows
query = f"""
DELETE FROM `{mobile_data_cleaned}`
WHERE net_provider IN (
    SELECT net_provider
    FROM `{mobile_data_cleaned}`
    GROUP BY net_provider
    HAVING COUNT(*) < 10
)
    """
# Execute the query
run_query(query)

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: final count of unique net_provider
query = f"""
SELECT
 COUNT(DISTINCT(net_provider)) final_unique_net_provider
FROM `{mobile_data_cleaned}`
    """

# Execute the query
query_df(query)

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: 
query = f"""
SELECT
 net_provider,
 COUNT(*) activity_count
FROM `{mobile_data_cleaned}`
GROUP BY 1
ORDER BY 2 DESC
LIMIT 20
"""

# Execute the query 
query_df(query)

**Operator**

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: original count of unique net_provider
query = f"""
SELECT
 COUNT(DISTINCT(operator)) original_unique_net_operator
FROM `{mobile_data_cleaned}`
    """

# Execute the query
query_df(query)

    - Standardize operator names

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: Update operator names
query = f"""
UPDATE `{mobile_data_cleaned}`
SET operator = CASE
    # WHEN UPPER(TRIM(operator)) = '3' THEN 'Three'
    WHEN UPPER(TRIM(operator)) LIKE '3%' THEN 'Three'
    # WHEN UPPER(TRIM(operator)) = 'BITEL' THEN 'Bytel'
    WHEN UPPER(TRIM(operator)) LIKE '%AIRTEL%' THEN 'Airtel'
    WHEN UPPER(TRIM(operator)) LIKE '%B%TEL%' THEN 'Bytel'
    WHEN UPPER(TRIM(operator)) LIKE '%BOUYGUES%' THEN 'Bouygues Telecom'
    WHEN UPPER(TRIM(operator)) LIKE 'BUSCANDO %' THEN 'Sense Servei'
    WHEN UPPER(TRIM(operator)) = 'CABLE MOVIL' THEN 'Cable Movil'
    WHEN UPPER(TRIM(operator)) = 'CABLEMOVIL' THEN 'Cable Movil'
    WHEN UPPER(TRIM(operator)) LIKE 'CLARO%' THEN 'Claro'
    WHEN UPPER(TRIM(operator)) LIKE 'CUBACEL%' THEN 'Cubacel'
    WHEN UPPER(TRIM(operator)) LIKE 'E-%' THEN 'EE'
    WHEN UPPER(TRIM(operator)) LIKE '%EMER%' THEN 'Nomes Trucades Emergencies'
    WHEN UPPER(TRIM(operator)) LIKE 'JAZZTEL%' THEN 'Jazztel'
    WHEN UPPER(TRIM(operator)) LIKE 'LOWI%' THEN 'Lowi'
    WHEN LEFT(UPPER(TRIM(operator)), 14) = 'FRANCE TELECOM' THEN 'France Telcom Espana SA'
    WHEN UPPER(TRIM(operator)) LIKE 'MASMOVIL%' THEN 'Masmovil'
    WHEN UPPER(TRIM(operator)) LIKE '%MOBILAND%' THEN 'Mobiland'
    WHEN UPPER(TRIM(operator)) LIKE 'MOVILNET%' THEN 'Movilnet'
    WHEN UPPER(TRIM(operator)) LIKE '%MO%ISTAR%' THEN 'Movistar'
    # WHEN UPPER(TRIM(operator)) LIKE 'MOBISTAR%' THEN 'Movistar'
    # WHEN UPPER(TRIM(operator)) LIKE '%MOBISTAR%' THEN 'Movistar'
    # WHEN UPPER(TRIM(operator)) LIKE 'MOVISTAR%' THEN 'Movistar'
    WHEN UPPER(TRIM(operator)) LIKE 'MTS%' THEN 'MTS'
    WHEN UPPER(TRIM(operator)) LIKE '%ORANGE%' THEN 'Orange'
    WHEN UPPER(TRIM(operator)) = 'A1' THEN 'Orange'
    WHEN UPPER(TRIM(operator)) LIKE 'O2%' THEN 'O2'
    WHEN UPPER(TRIM(operator)) LIKE 'PE%EPHONE' THEN 'Pepephone'
    # WHEN UPPER(TRIM(operator)) = 'PELEPHONE' THEN 'Pepephone'
    # WHEN UPPER(TRIM(operator)) = 'PEPEPHONE' THEN 'Pepephone'
    WHEN UPPER(TRIM(operator)) = 'PROXIMUS' THEN 'Proximus'
    WHEN UPPER(TRIM(operator)) LIKE 'REPUBLICA%' THEN 'Republica Movil'
    WHEN UPPER(TRIM(operator)) LIKE 'SENSE %' THEN 'Sense Servei'
    WHEN UPPER(TRIM(operator)) LIKE 'SIMYO%' THEN 'Simyo'
    WHEN UPPER(TRIM(operator)) LIKE 'SIN %' THEN 'Sense Servei'
    WHEN UPPER(TRIM(operator)) LIKE 'TIGO%' THEN 'TIGO'
    WHEN UPPER(TRIM(operator)) LIKE 'TIM%' THEN 'TIM'
    WHEN UPPER(TRIM(operator)) LIKE 'TELEF%' THEN 'Movistar'
    # WHEN (TRIM(network)) = 'Telefonica Moviles Espana' THEN 'Movistar'
    # WHEN (TRIM(network)) = 'Telef├│nica M├│viles Espa├▒a' THEN 'Movistar'
    WHEN UPPER(TRIM(operator)) LIKE 'TDC%' THEN 'TDC Mobile'
    WHEN UPPER(TRIM(operator)) LIKE 'TELEKOM%' THEN 'Telekom'
    WHEN UPPER(TRIM(operator)) LIKE '%TELENOR%' THEN 'Telenor'
    WHEN UPPER(TRIM(operator)) LIKE '%T-MOBILE%' THEN 'T-Mobile'
    WHEN UPPER(TRIM(operator)) LIKE 'VIVO%' THEN 'Vivo'
    WHEN UPPER(TRIM(operator)) LIKE '%VODAFONE%' THEN 'Vodafone'
    WHEN UPPER(TRIM(operator)) = 'VF ES' THEN 'Vodafone'
    WHEN UPPER(TRIM(operator)) LIKE 'VODACOM%' THEN 'Vodacom'
    WHEN UPPER(TRIM(operator)) LIKE 'YOIGO%' THEN 'Yoigo'
    ELSE operator
END
WHERE operator IS NOT NULL;
    """

# Execute the query
run_query(query)

    - Null values in operator

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: Count null values and calculate the percentage
query = f"""
WITH NetOperatorCount AS (
 SELECT
   COUNT(*) AS total_count_operator
 FROM `{mobile_data_cleaned}`
),
NullOperatorCount AS (
 SELECT
   COUNT(*) AS null_count_operator
 FROM `{mobile_data_cleaned}`
 WHERE
   operator IS NULL
   OR operator = 'null'
)

SELECT
 null_count_operator,
 CONCAT(ROUND((null_count_operator/total_count_operator)*100, 2), " %") AS perc_null_operator
FROM NetOperatorCount, NullOperatorCount
"""

# Execute the query
query_df(query)

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: final count of unique operator
query = f"""
SELECT
 COUNT(DISTINCT(net_provider)) final_unique_operator
FROM `{mobile_data_cleaned}`
    """

# Execute the query
query_df(query)

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: 
query = f"""
SELECT
 operator,
 COUNT(*) activity_count
FROM `{mobile_data_cleaned}`
GROUP BY 1
ORDER BY 2 DESC
LIMIT 20
"""

# Execute the query 
query_df(query)

**Net**

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: 
query = f"""
SELECT
 net,
 COUNT(*) activity_record
FROM `{mobile_data_cleaned}`
GROUP BY 1
"""

# Execute the query 
query_df(query)

    - Replace null values for 'Undefined Net'

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: 
query = f"""
UPDATE `{mobile_data_cleaned}`
SET net = CASE
  WHEN net IS NULL THEN 'Undefined net'
  ELSE net
END
WHERE net IS NULL
"""

# Execute the query 
run_query(query)

**Provider**

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: 
query = f"""
SELECT
 provider,
 COUNT(*) activity_record
FROM `{mobile_data_cleaned}`
GROUP BY 1
"""

# Execute the query 
query_df(query)

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: 
query = f"""
UPDATE `{mobile_data_cleaned}`
SET provider = CASE
    WHEN UPPER(TRIM(provider)) LIKE '%GPS%' THEN 'GPS'
    WHEN UPPER(TRIM(provider)) LIKE '%FUSED%' THEN 'Fused'
    WHEN UPPER(TRIM(provider)) LIKE '%NETWORK%' THEN 'Network'
    WHEN provider IS NULL THEN 'Undefined Provider'
    ELSE 'Undefined Provider'
END
WHERE provider IS NOT NULL
"""

# Execute the query 
run_query(query)

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: 
query = f"""
SELECT
 provider,
 COUNT(*) activity_record
FROM `{mobile_data_cleaned}`
GROUP BY 1
"""

# Execute the query 
query_df(query)

**Postal Code** and **Town Names**

This part ensures that there are no discrepancies between postal codes and town names, and removes rows with missing values in both columns.

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query:
query = f"""
SELECT
 COUNT(*) one_valid_one_null
FROM `{mobile_data_cleaned}`
WHERE
    (postal_code IS NULL AND town_name IS NOT NULL)
    OR (postal_code IS NOT NULL AND town_name IS NULL)
"""

# Execute the query
query_df(query)

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query:
query = f"""
SELECT
 COUNT(*) both_features_null
FROM `{mobile_data_cleaned}`
WHERE
    postal_code IS NULL AND town_name IS NULL
"""

# Execute the query
query_df(query)

    - This means that all null values in postal_code correspond to null values in `town_name. Therefore, we are going to drop all rows with null values in both features.

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: delete specific rows
query = f"""
DELETE FROM `{mobile_data_cleaned}`
WHERE postal_code IS NULL AND town_name IS NULL
"""

# Execute the query 
run_query(query)

**Download Speed** and **Upload Speed**

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query:
query = f"""
SELECT
 DISTINCT(downloadSpeed)
FROM `{mobile_data_cleaned}`
LIMIT 60
"""

# Execute the query
query_df(query)

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query:
query = f"""
SELECT
 DISTINCT(uploadSpeed)
FROM `{mobile_data_cleaned}`
LIMIT 60
"""

# Execute the query
query_df(query)

    - Both columns have no recorded data and only contain null values. Since they do not provide any information, we are going to drop both columns.

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: delete specific rows
query = f"""
ALTER TABLE `{mobile_data_cleaned}`
DROP COLUMN downloadSpeed,
DROP COLUMN uploadSpeed
"""

# Execute the query 
run_query(query)

**Description** and **Activity**

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query:
query = f"""
SELECT
 description,
 COUNT(*) activity_record
FROM `{mobile_data_cleaned}`
GROUP BY 1
LIMIT 60
"""

# Execute the query
query_df(query)

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query:
query = f"""
SELECT
 activity,
 COUNT(*) activity_record
FROM `{mobile_data_cleaned}`
GROUP BY 1
LIMIT 60
"""

# Execute the query
query_df(query)

    - Update null values as 'UNKNOWN' activity

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: update NULL values
query = f"""
UPDATE `{mobile_data_cleaned}`
SET activity = CASE 
  WHEN activity IS NULL THEN 'UNKNOWN' 
END
WHERE activity IS NULL
"""

# Execute the query 
run_query(query)

### Numerical Features

#### Summary Statistics

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query:
query = f"""
SELECT
 status,
 speed,
 precision,
 signal,
 satellites
FROM `{mobile_data_cleaned}`
LIMIT 10
"""

# Execute the query
query_df(query)

    - As 'status' values are directly related to the 'description' (categorical) values, we will drop the 'status' column.

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: delete specific rows
query = f"""
ALTER TABLE `{mobile_data_cleaned}`
DROP COLUMN status
"""

# Execute the query 
run_query(query)

Despite the option to load the entire dataset into a DataFrame and later apply the pandas .describe() function to obtain all the summary statistics for numerical features, we are going to manually calculate some of them using SQL commands.

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query:
query = f"""
SELECT
  'speed' AS metric,
  MIN(speed) AS min_value,
  CAST(MAX(speed) AS INT64) AS max_value, -- Cast as INT64
  ROUND(AVG(speed), 2) AS avg_value,
  CAST(ROUND(STDDEV_POP(speed), 2) AS INT64) AS std_value -- Cast as INT64
FROM `{mobile_data_cleaned}`

UNION ALL

SELECT
  'satellites' AS metric,
  MIN(satellites) AS min_value,
  CAST(MAX(satellites) AS INT64) AS max_value, -- Cast as INT64
  ROUND(AVG(satellites), 2) AS avg_value,
  CAST(ROUND(STDDEV_POP(satellites), 2) AS INT64) AS std_value -- Cast as INT64
FROM `{mobile_data_cleaned}`

UNION ALL

SELECT
  'precision' AS metric,
  MIN(precision) AS min_value,
  CAST(MAX(precision) AS INT64) AS max_value, -- Cast as INT64
  ROUND(AVG(precision), 2) AS avg_value,
  CAST(ROUND(STDDEV_POP(precision), 2) AS INT64) AS std_value -- Cast as INT64
FROM `{mobile_data_cleaned}`

UNION ALL

SELECT
  'signal' AS metric,
  MIN(signal) AS min_value,
  CAST(MAX(signal) AS INT64) AS max_value, -- Cast as INT64
  ROUND(AVG(signal), 2) AS avg_value,
  CAST(ROUND(STDDEV_POP(signal), 2) AS INT64) AS std_value -- Cast as INT64
FROM `{mobile_data_cleaned}`
"""

# Execute the query
query_df(query)

    - Handling 'precision' and 'satellites' possible outliers 

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: precision values ordered DESC to check maximum values
query = f"""
SELECT
 CAST(precision AS INT64) precision
FROM `{mobile_data_cleaned}`
ORDER BY 1 DESC
LIMIT 10
"""

# Execute the query
query_df(query)

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: satellites values ordered DESC to check maximum values
query = f"""
SELECT
 CAST(satellites AS INT64) satellites
FROM `{mobile_data_cleaned}`
ORDER BY 1 DESC
LIMIT 10
"""

# Execute the query
query_df(query)

    - Handling outliers in 'precision' colum

Now we are going to calculate various percentiles (25th, 50th, 75th, 90th, 95th, 99th) for the 'precission' column in the table and display their values.

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: satellites values ordered DESC to check maximum values
query = f"""
WITH Percentiles AS (
  SELECT
    APPROX_QUANTILES(precision, 100)[OFFSET(25)] AS percentile_25,
    APPROX_QUANTILES(precision, 100)[OFFSET(50)] AS median,
    APPROX_QUANTILES(precision, 100)[OFFSET(75)] AS percentile_75,
    APPROX_QUANTILES(precision, 100)[OFFSET(90)] AS percentile_90,
    APPROX_QUANTILES(precision, 100)[OFFSET(95)] AS percentile_95,
    APPROX_QUANTILES(precision, 100)[OFFSET(99)] AS percentile_99
  FROM `{mobile_data_cleaned}`
)
SELECT percentile_25, median, percentile_75, percentile_90, percentile_95, percentile_99
FROM Percentiles;
"""

# Execute the query
query_df(query)

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: 
query = f"""
SELECT
 COUNT(*) precision_g_t_121_count
FROM `{mobile_data_cleaned}`
WHERE precision > 121
"""

# Execute the query
query_df(query)

    - Percentile 99: the top 1% of values (106577 rows) have precision values greater than 121.

    - How would the removal of these outliers affect the data distribution, and would this impact be consistent across different provinces?

This following query calculates and compares outlier percentages in different provinces based on the 'precision' column, where values above 121 are considered outliers. It provides insights into how outliers are distributed among provinces.

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: 
query = f"""
SELECT
  province,
  COUNT(*) AS province_records,
  SUM(CASE WHEN precision > 121 THEN 1 ELSE 0 END) AS province_outliers,
  ROUND((SUM(CASE WHEN precision > 121 THEN 1 ELSE 0 END) / COUNT(*)) * 100, 2) AS percentage_outliers
FROM `{mobile_data_cleaned}`
GROUP BY 1
ORDER BY 1
"""

# Execute the query
query_df(query)

    - Barcelona, Girona and Tarragona would have a similar impact (around 1.05% of their data) if these outliers are removed. Only Lleida would have a smaller impact (0.44%).
    
    - We are going to proceed with the deletion of 'precision' values higher than 121.

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: delete outliers in 'precision' column
query = f"""
DELETE FROM `{mobile_data_cleaned}`
WHERE precision > 121
"""

# Execute the query
run_query(query)

    - As a result of removing the 'precision' outliers, the outlier in the 'satellites' column, which had a value of 11503299477926, has also been eliminated from the dataset.

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: satellites values ordered DESC to check maximum values
query = f"""
SELECT
 CAST(satellites AS INT64) satellites
FROM `{mobile_data_cleaned}`
ORDER BY 1 DESC
LIMIT 10
"""

# Execute the query
query_df(query)

### Save Cleaned Table

If required, the fully cleaned dataset could be stored under the name 'mobile_data_2015_2017_cleaned_final' for use in subsequent analysis and reporting.

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: 
query = f"""
CREATE OR REPLACE TABLE `bq-analyst-230590.project_cat_mobile_coverage_2015_2017.mobile_data_2015_2017_cleaned_final`
AS
SELECT *
FROM `{mobile_data_cleaned}`
"""

# Execute the query
# run_query(query)

### Cleaned Table

In [ ]:
# Datasets: {mobile_data_cleaned}

# SQL query: 
query = f"""
SELECT *
FROM `{mobile_data_cleaned}`
#LIMIT 20
"""

# Execute the query
cleaned_data = query_df(query)

# Display data
display(cleaned_data.head(), cleaned_data.tail(), cleaned_data.shape, cleaned_data.info())